# Modelo Para Previsão de Doenças Usando Registros Médicos Eletrônicos - Parte 3

<h2>Criação do Endpoint a partir dos serviços em Nuvem</h2>

## Imports 

https://sagemaker.readthedocs.io/en/stable/

In [2]:
# Imports
import os
import json
import sagemaker
import boto3
import numpy as np
import pandas as pd
from sagemaker.serializers import CSVSerializer
from sagemaker.inputs import TrainingInput
from sagemaker.predictor import Predictor
from sagemaker import get_execution_role

In [3]:
sagemaker.__version__

'2.95.0'

In [4]:
boto3.__version__

'1.24.12'

Agora irei utilizar uma serie de outras funções disponíveis pelo SageMaker.
* **TrainingInput**: Essa função serve para carregar os dados em um formato serializado que servirá de entrada para o modelo criado a partir do SageMaker.

O boto3 é um pacote que provê interface para interagir via python com os serviços da AWS. 
> Link oficial do pacote: https://pypi.org/project/boto/

## Carrega os Dados

In [5]:
# Obtém a sessão do SageMaker
session = boto3.Session()

s3 = session.resource('s3')
s3

s3.ServiceResource()

In [6]:
from sagemaker import get_execution_role
role = get_execution_role()
print(role)

arn:aws:iam::351371806175:role/service-role/AmazonSageMaker-ExecutionRole-20220722T092670


In [7]:
s3_bucket = 'krupck-bucket-bloodpressure'
prefix = 'dados'

In [8]:
raiz = 's3://{}/{}/'.format(s3_bucket, prefix)
print(raiz)

s3://krupck-bucket-bloodpressure/dados/


In [9]:
dados_treino = TrainingInput(s3_data = raiz + 'treino.csv', content_type = 'csv')
dados_teste = TrainingInput(s3_data = raiz + 'teste.csv', content_type = 'csv')

In [10]:
print(json.dumps(dados_treino.__dict__, indent = 2))

{
  "config": {
    "DataSource": {
      "S3DataSource": {
        "S3DataType": "S3Prefix",
        "S3Uri": "s3://krupck-bucket-bloodpressure/dados/treino.csv",
        "S3DataDistributionType": "FullyReplicated"
      }
    },
    "ContentType": "csv"
  }
}


In [11]:
print(json.dumps(dados_teste.__dict__, indent = 2))

{
  "config": {
    "DataSource": {
      "S3DataSource": {
        "S3DataType": "S3Prefix",
        "S3Uri": "s3://krupck-bucket-bloodpressure/dados/teste.csv",
        "S3DataDistributionType": "FullyReplicated"
      }
    },
    "ContentType": "csv"
  }
}


## Construção e Treinamento do Modelo

Vamos construir o modelo. Primeiro irei criar um container a partir da função image_uris passando as configurações que eu quero.

Um container é basicamente uma máquina virtual super leve criada a partir de uma imagem (template) Docker. Com esse container vamos utilizar uma máquina lá na nuvem da Amazon para realizar o treinamento. Quando acabar o treinamento, o container será destruído.

Como parâmetros do template temos:
* **region**: região que está configurada no Amazon SageMaker capturada a partid da sessão boto que criamos;
* **framework**: Algoritmo que será utilizado para o projeto: XGBoost;
* **version**: Versão do algoritmo: 1.0-1;
* **escopo do container**: Será um container para treinamento;

In [12]:
# Criação do Container
# https://sagemaker.readthedocs.io/en/stable/api/utility/image_uris.html
container_uri = sagemaker.image_uris.retrieve(region = session.region_name, 
                                              framework = 'xgboost', 
                                              version = '1.0-1', 
                                              image_scope = 'training')

In [13]:
# Argumentos do estimador
sagemaker_execution_role = role
sagemaker_session = sagemaker.Session()

Agora iremos criar o estimator que vai efetivamente realizar o treinamento, detalhando cada parâmetro:
* **image_uri**: É o container que foi criado anteriormente;
* **role**: É a execution role obtida no início do projeto;
* **instance_count**: Número de nodes (máquinas) que iremos utilizar para a execução do projeto;
* **instance_type**: A capacidade computacional de cada instância: https://aws.amazon.com/pt/sagemaker/pricing/?nc=sn&loc=3&refid=ft_card
* **output_path**: Diretório de saída que será armazenado o resultado do modelo;
* **base_job_name**: Nome do que iremos fazer 

In [14]:
# Criação do Estimador
# https://sagemaker.readthedocs.io/en/stable/api/training/estimators.html
xgb = sagemaker.estimator.Estimator(image_uri = container_uri,
                                    role = sagemaker_execution_role, 
                                    instance_count = 2, 
                                    instance_type = 'ml.m5.xlarge',
                                    output_path = 's3://{}/artefatos'.format(s3_bucket),
                                    sagemaker_session = sagemaker_session,
                                    base_job_name = 'classifier')

**Definição dos Hiperparâmetros**
<br/>
Agora a definição dos hiperparâmetros. Aqui estamos tratando em nível de algoritmo, vamos configurar os parâmetros para o algoritmo ser executado.

Definir a melhor combinação de hiperparâmetros é um grande desafio para o cientista. Testar todas as possibilidades pode levar muito tempo, em contrapartida, deixar de testar mais possibilidades podemos deixar de obter o máximo de precisão que o modelo poderia atingir. É um trade-off.

Para esse projeto o intuito é testar a computação em nuvem, portanto não irei me preocupar em testar várias combinações de parâmetros.

Link para a documentação: https://docs.aws.amazon.com/pt_br/sagemaker/latest/dg/xgboost_hyperparameters.html

In [16]:
# Definição dos Hiperparâmetros
xgb.set_hyperparameters(objective = 'binary:logistic', num_round = 100)

**Execução do treinamento**

Vamos utilizar o próprio método `fit` já velho conhecido de quem já mexe com Machine Learning. Mas aqui temos uma diferença, nós iremos passar 2 pares de chave-valores, passamos `treino` e `validação`.

In [17]:
# Treinamento
xgb.fit({'train': dados_treino, 'validation': dados_teste})

2022-07-22 13:32:08 Starting - Starting the training job...
2022-07-22 13:32:31 Starting - Preparing the instances for trainingProfilerReport-1658496728: InProgress
......
2022-07-22 13:33:36 Downloading - Downloading input data...
2022-07-22 13:34:01 Training - Downloading the training image.....INFO:sagemaker-containers:Imported framework sagemaker_xgboost_container.training
INFO:sagemaker-containers:Failed to parse hyperparameter objective value binary:logistic to Json.
Returning the value itself
INFO:sagemaker-containers:No GPUs detected (normal if no gpus installed)
INFO:sagemaker_xgboost_container.training:Running XGBoost Sagemaker in algorithm mode
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Determined delimiter of CSV input is ','
[13:34:45] 7199x4 matrix with 28796 entries loaded from /opt/ml/input/data/train?format=csv&label_column=0&delimiter=,
INFO:root:Determined delimiter of CSV input is ','
[13:34:45] 24

## Endpoint a Partir do Modelo

Vamos gerar o Endpoint a partir do modelo para que possamos utilizá-lo na pratica. Isto é, irei converter o modelo para um formato em que ele possa ser usado no dia a dia. Essa etapa é extremamente necessária para que posssamos utilizar o modelo que treinamos, seja para utilizar o Predictor ou criar uma API.

A nomenclatura Endpoint é amplamente utilizada pelo SageMaker, se consultarmos a documentação é exatamente esta nomenclatura que iremos encontrar.


In [18]:
# Deploy do modelo treinado criando o endpoint
# https://docs.aws.amazon.com/pt_br/sagemaker/latest/dg/xgboost.html
xgb_predictor = xgb.deploy(initial_instance_count = 2, instance_type = 'ml.m5.xlarge')

----!

## Previsões a Partir do Endpoint

Agora que temos o deploy do nosso modelo, podemos realizar previsões. Primeiro temos que criar um objeto da classe CSVSerializer. Este objeto vai permitir que serializamos o modelo para ser executado.

In [19]:
csv_serializer = CSVSerializer()

Agora irei instanciar um objeto da classe Predictor passando como parâmetros o `Endpoint` gerado no tópico anteior e o `CSVSerializer` que foi definido.

In [20]:
predictor = Predictor(endpoint_name = xgb_predictor.endpoint_name, serializer = csv_serializer)

Carregando os dados de teste direto do bucket S3.

In [21]:
df_teste = pd.read_csv(raiz + 'teste.csv', names = ['class', 'bmi', 'diastolic_bp_change', 'systolic_bp_change', 'respiratory_rate'])

In [22]:
df_teste.head()

,class,bmi,diastolic_bp_change,systolic_bp_change,respiratory_rate
0,0,0.454357,-0.085644,0.057265,-0.118441
1,0,-0.664451,0.413848,0.484622,0.909034
2,1,-1.075770,-1.483066,-1.557970,0.576501
3,0,0.314809,1.162859,-0.744431,0.962562
4,0,1.561199,-0.490421,0.019759,-1.230564


In [23]:
X = df_teste.sample(1)
X

,class,bmi,diastolic_bp_change,systolic_bp_change,respiratory_rate
2130,0,0.108698,0.90109,1.861043,-0.125463


In [24]:
X = X.values[0]
X[1:]

array([ 0.10869849,  0.90109004,  1.86104276, -0.1254631 ])

In [25]:
paciente = X[1:]
paciente

array([ 0.10869849,  0.90109004,  1.86104276, -0.1254631 ])

In [27]:
# Faz a previsão de um paciente
predicted_class_prob = predictor.predict(paciente).decode('utf-8')
if float(predicted_class_prob) < 0.5:
    print('Previsão = Não tem problema de pressão')
else:
    print('Previsão = Tem problema de pressão')
print()

Previsão = Não tem problema de pressão



## Avaliando o Modelo

Agora vamos avaliar o modelo. O objeto `predictor` do Amazon SageMaker espera receber como entrada um único registro, portanto, para realizarmos a avaliação do modelo, não podemos simplesmente passar o array de valores como parâmetro.

É necessário fazer um loop e prever registro por registro, em seguida, armazenar o resultado em um novo array.

In [28]:
# Previsão de todos os pacientes no dataset de teste
predictions = []
expected = []
correct = 0
for row in df_teste.values:
    expected_class = row[0]
    payload = row[1:]
    predicted_class_prob = predictor.predict(payload).decode('utf-8')
    predicted_class = 1
    if float(predicted_class_prob) < 0.5:
        predicted_class = 0  
    if predicted_class == expected_class:
        correct += 1
    predictions.append(predicted_class)
    expected.append(expected_class)

Como no notebook anterior chegamos a conclusão de que o dataset é simulado e não representa a realidade, eu irei simplesmente calcular a acurácia simples aqui só para ter um resultado de saída.

In [29]:
print('Acurácia = {:.2f}%'.format(correct/len(predictions) * 100))

Acurácia = 78.50%


#### Confusion Matrix

Exibindo o resultado por matriz de confusão.

In [30]:
expected = pd.Series(np.array(expected))
predictions = pd.Series(np.array(predictions))
pd.crosstab(expected, predictions, rownames = ['Actual'], colnames = ['Predicted'], margins = True)

Predicted,0,1,All
Actual,,,
0.0,1871,89,1960
1.0,427,13,440
All,2298,102,2400


<h2>Considerações Finais</h2>

Nessa etapa carreguei os dados a partir do Bucket S3, depois criei uma imagem Docker chamada de Container para a criação de uma máquina virtual leve.

Em seguida criei o Estimador com o algoritmo XGBoost e utilizando as máquinas disponibilizadas pela Amazon para realizar o treinamento. E defini os Hiperparâmetros para a realização do treinamento do modelo.

Após realizar o treinamento do modelo, criei o Endpoint para colocar o modelo em produção efetivamente.

Na próxima parte irei criar um notebook especificamente para detalhar sobre a criação do Endpoint com o intuito de criar uma API.